In [56]:
!pip install torchinfo
!pip install av

In [2]:
# get data
!wget https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/r3yjf35hzr-1.zip
!unzip r3yjf35hzr-1.zip -d ./
!mv 'Shoplifting Dataset (2022) - CV Laboratory MNNIT Allahabad' shopliftingdata
!unzip /content/shopliftingdata/Dataset.zip -d ./
!cd /content/Dataset
!pip install -U kora

--2024-05-12 08:55:24--  https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/r3yjf35hzr-1.zip
Resolving prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)... 3.5.65.69, 52.218.93.56, 3.5.66.161, ...
Connecting to prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)|3.5.65.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 760777897 (726M) [application/zip]
Saving to: ‘r3yjf35hzr-1.zip’

r3yjf35hzr-1.zip    100%[===================>] 725.53M  11.3MB/s    in 76s     

2024-05-12 08:56:42 (9.52 MB/s) - ‘r3yjf35hzr-1.zip’ saved [760777897/760777897]

Archive:  r3yjf35hzr-1.zip
  inflating: ./Shoplifting Dataset (2022) - CV Laboratory MNNIT Allahabad/Dataset.zip  
Archive:  /content/shopliftingdata/Dataset.zip
   creating: ./Dataset/
   creating: ./Dataset/Normal/
  inflating: ./Dataset/Normal/Normal (1).mp4  
 

In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Video Processing/finalProject/Shoplifting_vp')

In [4]:
# importların yapılması
import os
import numpy as np
import gc

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
from copy import deepcopy
from torchinfo import summary

import dataloader as dataloader
import utils as utils

import time
import importlib

In [42]:
importlib.reload(dataloader)
importlib.reload(utils)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
device

device(type='cpu')

In [7]:
## initialize dataloader

# Define your transform (data pre-processing) fınction
# Define your dataset with transform
transform = transforms.Compose([
    dataloader.ShopliftingPreprocessing(output_size=(120, 120))
])


In [8]:
# Define your dataset
dataset = dataloader.ShopliftingDataLoader(root_dir='Dataset', transform=transform)

In [13]:
dataset.samples[35]

('Dataset/Normal/Normal (41).mp4', 0)

In [18]:
dataset.__getitem__(35)[0].shape

(3, 120, 120, 30)

In [19]:
# Split the dataset
train_set, val_set, test_set = dataloader.split_dataset(dataset)

In [21]:
len(train_set), len(val_set), len(test_set)

(145, 18, 19)

In [22]:
# Create data loaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=8, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=8, shuffle=False)

In [51]:
gc.collect()

1982

In [54]:
class CNN3D(nn.Module):
    def __init__(self, t_dim=30, img_x=120, img_y=120, drop_p=0.4, fc_hidden1=1024, fc_hidden2=128, num_classes=2, send_device_fc=True):
        super(CNN3D, self).__init__()

        # set video dimension
        self.t_dim = t_dim
        self.img_x = img_x
        self.img_y = img_y
        self.send_device_fc = send_device_fc

        # fully connected layer hidden nodes
        self.fc_hidden1, self.fc_hidden2 = fc_hidden1, fc_hidden2
        self.drop_p = drop_p  # dropout probability
        self.num_classes = num_classes # Normal, Shoplifting
        self.ch1, self.ch2, self.ch3, self.ch4, self.ch5, self.ch6 = 8, 16, 32, 64, 96, 128
        self.k1, self.k2, self.k3, self.k4 = (3, 3, 3), (5, 5, 5), (7, 7, 7), (3, 1, 1)  # 3d kernel size
        self.s1, self.s2, self.s3 = (1, 1, 1), (2, 1, 1), (2, 2, 2)  # 3d strides
        self.pd1, self.pd2 = (0, 0, 0), (0, 0, 1)  # 3d padding

        # Layer 0 - Shape
        self.conv0_0_out = utils.conv3D_output_size((self.t_dim, self.img_x, self.img_y), self.pd1, self.k1, self.s1)
        print(self.conv0_0_out)
        self.conv0_1_out = utils.conv3D_output_size(self.conv0_0_out, self.pd1, self.k1, self.s1)
        print(self.conv0_1_out)
        self.conv0_2_out = utils.conv3D_output_size(self.conv0_1_out, self.pd1, self.k1, self.s1)
        print(self.conv0_2_out)
        self.pool0_3_out = utils.pooling_output_size(self.conv0_2_out, self.s3, (1, 1, 1), (0, 0, 0))
        print("mp")
        print(self.pool0_3_out)
        self.conv0_4_out = utils.conv3D_output_size(self.pool0_3_out, self.pd1, self.k1, self.s1)
        print(self.conv0_4_out)
        self.pool0_5_out = utils.pooling_output_size(self.conv0_4_out, self.s2, (1, 1, 1), (0, 0, 0))
        print("mp")
        print(self.pool0_5_out)
        self.conv0_6_out = utils.conv3D_output_size(self.pool0_5_out, self.pd1, self.k1, self.s1)
        print(self.conv0_6_out)
        self.conv0_7_out = utils.conv3D_output_size(self.conv0_6_out, self.pd1, self.k2, self.s1)
        print(self.conv0_7_out)
        self.conv0_8_out = utils.conv3D_output_size(self.conv0_7_out, self.pd1, self.k1, self.s1)
        print(self.conv0_8_out)
        self.conv0_9_out = utils.conv3D_output_size(self.conv0_8_out, self.pd1, self.k1, self.s1)
        print(self.conv0_9_out)

        # Layer 0 - Shape Cont.
        self.conv0_12_out = utils.conv3D_output_size(self.conv0_9_out, self.pd1, self.k1, self.s1)
        print(self.conv0_12_out)
        self.conv0_13_out = utils.conv3D_output_size(self.conv0_12_out, self.pd1, self.k1, self.s1)
        print(self.conv0_13_out)
        self.pool0_14_out = utils.pooling_output_size(self.conv0_13_out, self.s3, (1, 1, 1), (0, 0, 0))
        print("mp")
        print(self.pool0_14_out)

        # Final Layer - Shape
        self.conv3_2_out = utils.conv3D_output_size(self.pool0_14_out, self.pd1, self.k1, self.s1)
        print(self.conv3_2_out)
        self.conv3_3_out = utils.conv3D_output_size(self.conv3_2_out, self.pd1, self.k1, self.s1)
        print(self.conv3_3_out)

        # Layer 0
        self.conv0_0 = nn.Conv3d(in_channels=3, out_channels=self.ch1, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)

        self.conv0_1 = nn.Conv3d(in_channels=self.ch1, out_channels=self.ch2, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)

        self.conv0_2 = nn.Conv3d(in_channels=self.ch2, out_channels=self.ch2, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)
        self.pool0_3 = nn.MaxPool3d(kernel_size = (2, 2, 2), stride = (1, 1, 1))
        self.conv0_4 = nn.Conv3d(in_channels=self.ch2, out_channels=self.ch2, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)
        self.conv0_4_2 = nn.Conv3d(in_channels=self.ch2, out_channels=self.ch3, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)
        self.pool0_5 = nn.MaxPool3d(kernel_size = (2, 1, 1), stride = (2, 1, 1)) # start point of parallel layers
        self.conv0_6 = nn.Conv3d(in_channels=self.ch2, out_channels=self.ch3, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)
        self.conv0_7 = nn.Conv3d(in_channels=self.ch3, out_channels=self.ch3, kernel_size=self.k2, stride=self.s1,
                               padding=self.pd1)
        self.conv0_8 = nn.Conv3d(in_channels=self.ch3, out_channels=self.ch3, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)
        self.conv0_9 = nn.Conv3d(in_channels=self.ch3, out_channels=self.ch3, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)
        print("Layer 0")
        # 1st parallel layer
        self.conv1_0 = nn.Conv3d(in_channels=self.ch2, out_channels=self.ch3, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)
        self.conv1_1 = nn.Conv3d(in_channels=self.ch3, out_channels=self.ch3, kernel_size=self.k3, stride=self.s1,
                               padding=self.pd1)
        self.conv1_2 = nn.Conv3d(in_channels=self.ch3, out_channels=self.ch3, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)

        print("Layer 1")
        # first concatination and batch normalisation
        self.bn0_11 = nn.BatchNorm3d(self.ch4)
        self.conv0_12 = nn.Conv3d(in_channels=self.ch4, out_channels=self.ch4, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)
        self.conv0_13 = nn.Conv3d(in_channels=self.ch4, out_channels=self.ch4, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)
        self.pool0_14 = nn.MaxPool3d(kernel_size = (2, 2, 2), stride = (2, 2, 2))
        print("First Concat")

        # 2nd parallel layer
        self.conv2_0 = nn.Conv3d(in_channels=self.ch3, out_channels=self.ch3, kernel_size=self.k1, stride=self.s3,
                               padding=self.pd1)
        self.pool2_1 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(1, 1, 1))
        self.conv2_2 = nn.Conv3d(in_channels=self.ch3, out_channels=self.ch4, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)
        self.conv2_3 = nn.Conv3d(in_channels=self.ch4, out_channels=self.ch4, kernel_size=self.k1, stride=self.s2,
                               padding=self.pd1)
        self.conv2_4 = nn.Conv3d(in_channels=self.ch4, out_channels=self.ch4, kernel_size=self.k4, stride=self.s2,
                               padding=self.pd1)
        self.pool2_5 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(1, 1, 1))
        print("2nd Parallel")

        # last sequential layer
        self.bn3_1 = nn.BatchNorm3d(self.ch4)
        self.conv3_2 = nn.Conv3d(in_channels=self.ch4, out_channels=self.ch5, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd2) # need to keep 3 time_dim
        self.conv3_3 = nn.Conv3d(in_channels=self.ch5, out_channels=self.ch5, kernel_size=self.k1, stride=self.s1,
                               padding=self.pd1)
        self.bn3_4 = nn.BatchNorm3d(self.ch5)
        print("Last Sequential")

        # Fully connected layes
        print(self.ch5)
        print(self.conv3_3_out[0])
        print(self.conv3_3_out[1])
        print(self.conv3_3_out[2])

        self.fc2 = nn.Linear(self.fc_hidden1, self.fc_hidden2)
        self.fc3 = nn.Linear(self.fc_hidden2, self.num_classes) # fully connected layer, output = binary_classes

    def forward(self, x_3d):
        # Layer 0
        a = self.conv0_0(x_3d)
        print(a.size())
        a = self.conv0_1(a)
        print(a.size())
        a = self.conv0_2(a)
        print(a.size())
        print("mp")
        a = self.pool0_3(a)
        print(a.size())
        a = self.conv0_4(a)
        print(a.size())
        c = self.conv0_4_2(a)
        print(c.size())
        a = self.pool0_5(a)
        print("mp")
        print(a.size())
        b = a
        a = self.conv0_6(a)
        print(a.size())
        a = self.conv0_7(a)
        print(a.size())
        a = self.conv0_8(a)
        print(a.size())
        a = self.conv0_9(a)
        print(a.size())

        # Layer 1
        print("b")
        b = self.conv1_0(b)
        print(b.size())
        b = self.conv1_1(b)
        print(b.size())
        b = self.conv1_2(b)
        print(b.size())

        # Layer 0 - 1 merge
        print("ab")
        ab = torch.cat([a, b], dim=1)
        print(ab.size())
        ab = self.bn0_11(ab)
        print(ab.size())
        ab = self.conv0_12(ab)
        print(ab.size())
        ab = self.conv0_13(ab)
        print(ab.size())
        ab = self.pool0_14(ab)
        print(ab.size())

        # Layer 2
        print("c")
        c = self.conv2_0(c)
        print(c.size())
        c = self.pool2_1(c)
        print(c.size())
        c = self.conv2_2(c)
        print(c.size())
        c = self.conv2_3(c)
        print(c.size())
        c = self.conv2_4(c)
        print(c.size())
        c = self.pool2_5(c)
        print(c.size())

        # Layer 0 - 1 - 2 merge
        print("abc")
        abc = torch.cat([ab, c], dim=2)
        print(abc.size())
        abc = self.bn3_1(abc)
        print(abc.size())
        abc = self.conv3_2(abc)
        print(abc.size())
        abc = self.conv3_3(abc)
        print(abc.size())
        abc = self.bn3_4(abc)
        print(abc.size())


        # Dropout & fully connected layers
        abc = abc.view(abc.size(0), -1)
        print(abc.size())
        abc = F.dropout(abc, p=self.drop_p, training=self.training)
        print(abc.size()[1])
        if self.send_device_fc: # Handler for GPU-CPU confusion
          fc1 = nn.Linear(abc.size()[1], self.fc_hidden1) # fully connected hidden layer
          fc1.to(device)
          abc = fc1(abc)
        else:
          abc = nn.Linear(abc.size()[1], self.fc_hidden1)(abc)  # fully connected hidden layer

        print(abc.size())
        abc = F.dropout(abc, p=self.drop_p, training=self.training)
        print(abc.size())
        abc = self.fc2(abc)
        print(abc.size())
        abc = F.dropout(abc, p=self.drop_p, training=self.training)
        print(abc.size())
        abc = self.fc3(abc)
        print(abc.size())
        output = F.softmax(abc, dim=1)
        print(output.size())

        return output

In [55]:
# Let's create an instance of the model and print its architecture
model = CNN3D(t_dim=30, img_x=120, img_y=120, drop_p=0.4, fc_hidden1=1024, fc_hidden2=128, num_classes=2, send_device_fc=False)
print(model)

# Define the shape of the dummy input data (batch_size, channels, frames, height, width)
dummy_input_shape = (32, 3, 120, 120, 30) # Batch size 32, 3 input channels, 16 frames, 112x112 resolution

# Generate random dummy input data within the defined shape
dummy_input = torch.randn(*dummy_input_shape)

# Forward pass through the model
output = model(dummy_input)

# Print the output shape
print("Output shape:", output.shape)

(28, 118, 118)
(26, 116, 116)
(24, 114, 114)
mp
(23, 113, 113)
(21, 111, 111)
mp
(20, 111, 111)
(18, 109, 109)
(14, 105, 105)
(12, 103, 103)
(10, 101, 101)
(8, 99, 99)
(6, 97, 97)
mp
(5, 96, 96)
(3, 94, 94)
(1, 92, 92)
Layer 0
Layer 1
First Concat
2nd Parallel
Last Sequential
96
1
92
92
CNN3D(
  (conv0_0): Conv3d(3, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv0_1): Conv3d(8, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv0_2): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool0_3): MaxPool3d(kernel_size=(2, 2, 2), stride=(1, 1, 1), padding=0, dilation=1, ceil_mode=False)
  (conv0_4): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv0_4_2): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool0_5): MaxPool3d(kernel_size=(2, 1, 1), stride=(2, 1, 1), padding=0, dilation=1, ceil_mode=False)
  (conv0_6): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv0_7): Conv3d(32, 32, kernel_size=(5, 5, 5), stride=(1, 1, 1))
  (conv0_8): Co

In [50]:
summary(model)

Layer (type:depth-idx)                   Param #
CNN3D                                    --
├─Conv3d: 1-1                            656
├─Conv3d: 1-2                            3,472
├─Conv3d: 1-3                            6,928
├─MaxPool3d: 1-4                         --
├─Conv3d: 1-5                            6,928
├─Conv3d: 1-6                            13,856
├─MaxPool3d: 1-7                         --
├─Conv3d: 1-8                            13,856
├─Conv3d: 1-9                            128,032
├─Conv3d: 1-10                           27,680
├─Conv3d: 1-11                           27,680
├─Conv3d: 1-12                           13,856
├─Conv3d: 1-13                           351,264
├─Conv3d: 1-14                           27,680
├─BatchNorm3d: 1-15                      128
├─Conv3d: 1-16                           110,656
├─Conv3d: 1-17                           110,656
├─MaxPool3d: 1-18                        --
├─Conv3d: 1-19                           27,680
├─MaxPool3d:

In [ ]:
model = CNN3D(t_dim=30, img_x=112, img_y=112, drop_p=0.4, fc_hidden1=1024, fc_hidden2=128, num_classes=2, send_device_fc=True)

(28, 110, 110)
(26, 108, 108)
(24, 106, 106)
(23, 105, 105)
(21, 103, 103)
(20, 103, 103)
(18, 101, 101)
(14, 97, 97)
(12, 95, 95)
(10, 93, 93)
(8, 91, 91)
(6, 89, 89)
(5, 88, 88)
(3, 86, 86)
(1, 84, 84)
Layer 0
Layer 1
First Concat
2nd Parallel
Last Sequential
96
1
84
84


In [ ]:
# hyper-params
num_epochs = 100
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# dont forget to send model to device
model.to(device)

CNN3D(
  (conv0_0): Conv3d(3, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv0_1): Conv3d(8, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv0_2): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool0_3): MaxPool3d(kernel_size=(2, 2, 2), stride=(1, 1, 1), padding=0, dilation=1, ceil_mode=False)
  (conv0_4): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv0_4_2): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool0_5): MaxPool3d(kernel_size=(2, 1, 1), stride=(2, 1, 1), padding=0, dilation=1, ceil_mode=False)
  (conv0_6): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv0_7): Conv3d(32, 32, kernel_size=(5, 5, 5), stride=(1, 1, 1))
  (conv0_8): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv0_9): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv1_0): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (conv1_1): Conv3d(32, 32, kernel_size=(7, 7, 7), stride=(1, 1, 1))
  (conv1_2): Conv3d(32, 32,

In [ ]:
# Training loop
best_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device, dtype=torch.float32), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        print(outputs)
        print(targets)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device, dtype=torch.float32), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    if (100 * correct / total) > best_acc:
        best_acc = 100 * correct / total
        best_model_wts = deepcopy(model.state_dict())

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {running_loss / len(train_loader.dataset):.4f}, "
          f"Val Loss: {val_loss / len(val_loader.dataset):.4f}, "
          f"Val Acc: {(100 * correct / total):.2f}%")

# Testing loop
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device, dtype=torch.float32), targets.to(device)
        print(outputs)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

print(f"Test Loss: {test_loss / len(test_loader.dataset):.4f}, "
      f"Test Acc: {(100 * correct / total):.2f}%")

/usr/local/lib/python3.10/dist-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


tensor([[0.7868, 0.2132],
        [0.6745, 0.3255],
        [0.8398, 0.1602],
        [0.8308, 0.1692],
        [0.7429, 0.2571],
        [0.7350, 0.2650],
        [0.4778, 0.5222],
        [0.5581, 0.4419],
        [0.7224, 0.2776],
        [0.6286, 0.3714],
        [0.7884, 0.2116],
        [0.6537, 0.3463],
        [0.5347, 0.4653],
        [0.6219, 0.3781],
        [0.8091, 0.1909],
        [0.7326, 0.2674]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
tensor([1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1], device='cuda:0')


ValueError: Target size (torch.Size([16])) must be the same as input size (torch.Size([16, 2]))